In [8]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import warnings

# Suppress specific warnings from FastF1 and others if needed
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Enable FastF1 caching
try:
    fastf1.Cache.enable_cache("f1_cache")
except Exception as e:
    print(f"Error enabling cache: {e}. Check cache directory permissions.")

def get_driver_features(driver_code):
    """Return driver-specific features based on driver code."""
    # Driver characteristics dictionary
    # Values are normalized between 0-1 where appropriate
    driver_characteristics = {
        # Driver Code: {features}
        # Current drivers (2025 season)
        'VER': {'WetWeatherSkill': 0.95, 'QualifyingPace': 0.95, 'RaceCraft': 0.95, 'Consistency': 0.90, 'Aggression': 0.90, 'TireManagement': 0.85},
        'NOR': {'WetWeatherSkill': 0.85, 'QualifyingPace': 0.92, 'RaceCraft': 0.88, 'Consistency': 0.88, 'Aggression': 0.75, 'TireManagement': 0.87},
        'PIA': {'WetWeatherSkill': 0.82, 'QualifyingPace': 0.88, 'RaceCraft': 0.84, 'Consistency': 0.85, 'Aggression': 0.78, 'TireManagement': 0.85},
        'LEC': {'WetWeatherSkill': 0.82, 'QualifyingPace': 0.95, 'RaceCraft': 0.85, 'Consistency': 0.80, 'Aggression': 0.85, 'TireManagement': 0.80},
        'HAM': {'WetWeatherSkill': 0.95, 'QualifyingPace': 0.90, 'RaceCraft': 0.95, 'Consistency': 0.92, 'Aggression': 0.80, 'TireManagement': 0.90},
        'RUS': {'WetWeatherSkill': 0.80, 'QualifyingPace': 0.90, 'RaceCraft': 0.85, 'Consistency': 0.85, 'Aggression': 0.83, 'TireManagement': 0.82},
        'ANT': {'WetWeatherSkill': 0.78, 'QualifyingPace': 0.85, 'RaceCraft': 0.78, 'Consistency': 0.76, 'Aggression': 0.85, 'TireManagement': 0.75},
        'SAI': {'WetWeatherSkill': 0.84, 'QualifyingPace': 0.88, 'RaceCraft': 0.87, 'Consistency': 0.87, 'Aggression': 0.82, 'TireManagement': 0.85},
        'ALO': {'WetWeatherSkill': 0.92, 'QualifyingPace': 0.90, 'RaceCraft': 0.95, 'Consistency': 0.90, 'Aggression': 0.88, 'TireManagement': 0.90},
        'STR': {'WetWeatherSkill': 0.85, 'QualifyingPace': 0.75, 'RaceCraft': 0.80, 'Consistency': 0.75, 'Aggression': 0.75, 'TireManagement': 0.78},
        'TSU': {'WetWeatherSkill': 0.78, 'QualifyingPace': 0.85, 'RaceCraft': 0.82, 'Consistency': 0.75, 'Aggression': 0.90, 'TireManagement': 0.78},
        'HAD': {'WetWeatherSkill': 0.75, 'QualifyingPace': 0.80, 'RaceCraft': 0.78, 'Consistency': 0.74, 'Aggression': 0.85, 'TireManagement': 0.74},
        'ALB': {'WetWeatherSkill': 0.80, 'QualifyingPace': 0.85, 'RaceCraft': 0.84, 'Consistency': 0.82, 'Aggression': 0.75, 'TireManagement': 0.83},
        'GAS': {'WetWeatherSkill': 0.83, 'QualifyingPace': 0.84, 'RaceCraft': 0.85, 'Consistency': 0.83, 'Aggression': 0.80, 'TireManagement': 0.82},
        'OCO': {'WetWeatherSkill': 0.80, 'QualifyingPace': 0.82, 'RaceCraft': 0.83, 'Consistency': 0.80, 'Aggression': 0.85, 'TireManagement': 0.78},
        'HUL': {'WetWeatherSkill': 0.85, 'QualifyingPace': 0.82, 'RaceCraft': 0.80, 'Consistency': 0.83, 'Aggression': 0.75, 'TireManagement': 0.80},
        'BOR': {'WetWeatherSkill': 0.74, 'QualifyingPace': 0.76, 'RaceCraft': 0.76, 'Consistency': 0.74, 'Aggression': 0.82, 'TireManagement': 0.75},
        'DOO': {'WetWeatherSkill': 0.74, 'QualifyingPace': 0.78, 'RaceCraft': 0.75, 'Consistency': 0.73, 'Aggression': 0.85, 'TireManagement': 0.74},
        'BEA': {'WetWeatherSkill': 0.75, 'QualifyingPace': 0.78, 'RaceCraft': 0.77, 'Consistency': 0.75, 'Aggression': 0.80, 'TireManagement': 0.76},
        'LAW': {'WetWeatherSkill': 0.76, 'QualifyingPace': 0.79, 'RaceCraft': 0.78, 'Consistency': 0.75, 'Aggression': 0.85, 'TireManagement': 0.75},
        # Historical drivers (for reference)
        '44': {'WetWeatherSkill': 0.95, 'QualifyingPace': 0.90, 'RaceCraft': 0.95, 'Consistency': 0.92, 'Aggression': 0.80, 'TireManagement': 0.90},
        '33': {'WetWeatherSkill': 0.95, 'QualifyingPace': 0.95, 'RaceCraft': 0.95, 'Consistency': 0.90, 'Aggression': 0.90, 'TireManagement': 0.85},
        '16': {'WetWeatherSkill': 0.82, 'QualifyingPace': 0.95, 'RaceCraft': 0.85, 'Consistency': 0.80, 'Aggression': 0.85, 'TireManagement': 0.80},
        '55': {'WetWeatherSkill': 0.84, 'QualifyingPace': 0.88, 'RaceCraft': 0.87, 'Consistency': 0.87, 'Aggression': 0.82, 'TireManagement': 0.85},
    }
    
    # Default values if driver not found
    default_features = {
        'WetWeatherSkill': 0.75, 
        'QualifyingPace': 0.75, 
        'RaceCraft': 0.75, 
        'Consistency': 0.75, 
        'Aggression': 0.75, 
        'TireManagement': 0.75
    }
    
    # Return driver features if found, otherwise default
    if driver_code in driver_characteristics:
        return driver_characteristics[driver_code]
    # Try to handle numeric driver codes from historical data
    elif str(driver_code) in driver_characteristics:
        return driver_characteristics[str(driver_code)]
    else:
        print(f"Driver characteristics not found for '{driver_code}'. Using default values.")
        return default_features

def get_track_features(circuit_name):
    """Return track-specific features based on circuit name."""
    # Track characteristics dictionary
    # Values are normalized between 0-1 where appropriate
    track_characteristics = {
        # Circuit Name: {features}
        'Abu Dhabi': {'HighDownforce': 0.7, 'HighSpeed': 0.6, 'TireWear': 0.5, 'OvertakingDifficulty': 0.7},
        'Australia': {'HighDownforce': 0.6, 'HighSpeed': 0.7, 'TireWear': 0.6, 'OvertakingDifficulty': 0.6},
        'Austria': {'HighDownforce': 0.4, 'HighSpeed': 0.7, 'TireWear': 0.7, 'OvertakingDifficulty': 0.4},
        'Azerbaijan': {'HighDownforce': 0.3, 'HighSpeed': 0.8, 'TireWear': 0.4, 'OvertakingDifficulty': 0.5},
        'Bahrain': {'HighDownforce': 0.5, 'HighSpeed': 0.6, 'TireWear': 0.8, 'OvertakingDifficulty': 0.4},
        'Belgium': {'HighDownforce': 0.4, 'HighSpeed': 0.9, 'TireWear': 0.7, 'OvertakingDifficulty': 0.3},
        'Brazil': {'HighDownforce': 0.6, 'HighSpeed': 0.7, 'TireWear': 0.7, 'OvertakingDifficulty': 0.4},
        'Canada': {'HighDownforce': 0.6, 'HighSpeed': 0.6, 'TireWear': 0.7, 'OvertakingDifficulty': 0.5},
        'China': {'HighDownforce': 0.6, 'HighSpeed': 0.7, 'TireWear': 0.8, 'OvertakingDifficulty': 0.5},
        'Emilia Romagna': {'HighDownforce': 0.7, 'HighSpeed': 0.5, 'TireWear': 0.6, 'OvertakingDifficulty': 0.8},
        'France': {'HighDownforce': 0.6, 'HighSpeed': 0.7, 'TireWear': 0.6, 'OvertakingDifficulty': 0.6},
        'Great Britain': {'HighDownforce': 0.8, 'HighSpeed': 0.9, 'TireWear': 0.8, 'OvertakingDifficulty': 0.6},
        'Hungary': {'HighDownforce': 0.9, 'HighSpeed': 0.4, 'TireWear': 0.7, 'OvertakingDifficulty': 0.9},
        'Italy': {'HighDownforce': 0.3, 'HighSpeed': 0.9, 'TireWear': 0.5, 'OvertakingDifficulty': 0.5},
        'Japan': {'HighDownforce': 0.8, 'HighSpeed': 0.8, 'TireWear': 0.7, 'OvertakingDifficulty': 0.7},
        'Las Vegas': {'HighDownforce': 0.4, 'HighSpeed': 0.8, 'TireWear': 0.5, 'OvertakingDifficulty': 0.5},
        'Mexico': {'HighDownforce': 0.8, 'HighSpeed': 0.6, 'TireWear': 0.6, 'OvertakingDifficulty': 0.7},
        'Miami': {'HighDownforce': 0.5, 'HighSpeed': 0.7, 'TireWear': 0.6, 'OvertakingDifficulty': 0.6},
        'Monaco': {'HighDownforce': 0.9, 'HighSpeed': 0.2, 'TireWear': 0.3, 'OvertakingDifficulty': 1.0},
        'Netherlands': {'HighDownforce': 0.8, 'HighSpeed': 0.6, 'TireWear': 0.6, 'OvertakingDifficulty': 0.8},
        'Qatar': {'HighDownforce': 0.6, 'HighSpeed': 0.8, 'TireWear': 0.7, 'OvertakingDifficulty': 0.5},
        'Saudi Arabia': {'HighDownforce': 0.4, 'HighSpeed': 0.9, 'TireWear': 0.6, 'OvertakingDifficulty': 0.6},
        'Singapore': {'HighDownforce': 0.9, 'HighSpeed': 0.3, 'TireWear': 0.6, 'OvertakingDifficulty': 0.9},
        'Spain': {'HighDownforce': 0.7, 'HighSpeed': 0.6, 'TireWear': 0.8, 'OvertakingDifficulty': 0.7},
        'United States': {'HighDownforce': 0.6, 'HighSpeed': 0.7, 'TireWear': 0.7, 'OvertakingDifficulty': 0.5},
        'Styrian': {'HighDownforce': 0.4, 'HighSpeed': 0.7, 'TireWear': 0.7, 'OvertakingDifficulty': 0.4},
        'Sakhir': {'HighDownforce': 0.5, 'HighSpeed': 0.6, 'TireWear': 0.8, 'OvertakingDifficulty': 0.4},
        'Portuguese': {'HighDownforce': 0.7, 'HighSpeed': 0.6, 'TireWear': 0.6, 'OvertakingDifficulty': 0.6},
        'Tuscan': {'HighDownforce': 0.7, 'HighSpeed': 0.7, 'TireWear': 0.7, 'OvertakingDifficulty': 0.7},
        'Eifel': {'HighDownforce': 0.6, 'HighSpeed': 0.6, 'TireWear': 0.6, 'OvertakingDifficulty': 0.6},
        'Russian': {'HighDownforce': 0.5, 'HighSpeed': 0.7, 'TireWear': 0.5, 'OvertakingDifficulty': 0.6},
        '70th Anniversary': {'HighDownforce': 0.8, 'HighSpeed': 0.9, 'TireWear': 0.9, 'OvertakingDifficulty': 0.6},
        'São Paulo': {'HighDownforce': 0.6, 'HighSpeed': 0.7, 'TireWear': 0.7, 'OvertakingDifficulty': 0.4},
    }
    
    # Default values if track not found
    default_features = {
        'HighDownforce': 0.6, 
        'HighSpeed': 0.6, 
        'TireWear': 0.6, 
        'OvertakingDifficulty': 0.6
    }
    
    # Search for the circuit in our dictionary (partial match)
    for track_name, features in track_characteristics.items():
        if track_name in circuit_name:
            return features
    
    print(f"Track characteristics not found for '{circuit_name}'. Using default values.")
    return default_features

def get_standings_before_round(year, target_round):
    """Calculate driver and constructor points standings *before* a specific round."""
    driver_standings = {}
    constructor_standings = {}
    max_driver_points = 0
    max_constructor_points = 0
    try:
        schedule = fastf1.get_event_schedule(year, include_testing=False)
        races_before = schedule[schedule['RoundNumber'] < target_round]

        if races_before.empty:
            return {}, 0, {}, 0 # No races before this one

        for index, race in races_before.iterrows():
            try:
                session = fastf1.get_session(year, race['EventName'], 'R')
                session.load(laps=False, telemetry=False, weather=False, messages=False)
                results = session.results
                if results is not None and not results.empty:
                    for _, driver_result in results.iterrows():
                        driver = driver_result['Abbreviation']
                        team = driver_result['TeamName']
                        points = driver_result['Points']
                        # Update driver standings
                        driver_standings[driver] = driver_standings.get(driver, 0) + points
                        # Update constructor standings
                        constructor_standings[team] = constructor_standings.get(team, 0) + points
            except Exception as e:
                # Silently continue if a past race fails to load
                continue

        if driver_standings:
            max_driver_points = max(driver_standings.values()) if driver_standings else 0
        if constructor_standings:
             max_constructor_points = max(constructor_standings.values()) if constructor_standings else 0

    except Exception as e:
        print(f"Error calculating standings for {year} before round {target_round}: {e}")
        return {}, 0, {}, 0

    return driver_standings, max_driver_points, constructor_standings, max_constructor_points

def get_race_data(year, event_identifier):
    """Get Race and Qualifying data for a specific event."""
    try:
        session_r = fastf1.get_session(year, event_identifier, 'R')
        session_r.load(laps=False, telemetry=False, weather=False, messages=False)
        results_r = session_r.results

        if results_r is None or results_r.empty:
            print(f"No race results found for {year} {event_identifier}. Skipping.")
            return None

        results_r['FinishingPosition'] = pd.to_numeric(results_r['Position'], errors='coerce')
        results_r['GridPosition'] = pd.to_numeric(results_r['GridPosition'], errors='coerce')  # Add GridPosition
        results_r.dropna(subset=['FinishingPosition', 'GridPosition'], inplace=True)  # Ensure both positions are valid
        results_r[['FinishingPosition', 'GridPosition']] = results_r[['FinishingPosition', 'GridPosition']].astype(int)

        session_q = fastf1.get_session(year, event_identifier, 'Q')
        session_q.load(laps=False, telemetry=False, weather=False, messages=False)
        results_q = session_q.results

        if results_q is None or results_q.empty:
            print(f"No qualifying results found for {year} {event_identifier}. Proceeding without Quali times.")
            qualifying_times = pd.DataFrame({'Abbreviation': results_r['Abbreviation'], 'QualifyingTime (s)': np.nan})
        else:
            results_q['QualifyingTime'] = results_q[['Q1', 'Q2', 'Q3']].min(axis=1)
            results_q['QualifyingTime (s)'] = results_q['QualifyingTime'].dt.total_seconds()
            qualifying_times = results_q[['Abbreviation', 'QualifyingTime (s)']].copy()
        
        race_data = pd.merge(results_r[['Abbreviation', 'TeamName', 'FinishingPosition', 'GridPosition']], 
                             qualifying_times, on='Abbreviation', how='left')
        
        race_data['Year'] = year
        race_data['RoundNumber'] = session_r.event['RoundNumber']
        race_data['EventName'] = session_r.event['EventName']
        
        # Add track characteristics
        circuit_name = session_r.event['EventName']
        track_features = get_track_features(circuit_name)
        
        for feature_name, feature_value in track_features.items():
            race_data[feature_name] = feature_value
        
        # Add driver characteristics
        for idx, row in race_data.iterrows():
            driver_code = row['Abbreviation']
            driver_features = get_driver_features(driver_code)
            for feature_name, feature_value in driver_features.items():
                race_data.at[idx, feature_name] = feature_value
        
        # Calculate Points Index *before* this race using the updated function
        driver_standings, max_driver_points, constructor_standings, max_constructor_points = get_standings_before_round(year, race_data['RoundNumber'].iloc[0])
        
        # Calculate Driver Points Index
        if max_driver_points > 0:
            race_data['PointsIndex'] = race_data['Abbreviation'].apply(lambda x: driver_standings.get(x, 0) / max_driver_points)
        else:
            race_data['PointsIndex'] = 0.0
            
        # Calculate Constructor Points Index
        if max_constructor_points > 0:
            race_data['ConstructorPointsIndex'] = race_data['TeamName'].apply(lambda x: constructor_standings.get(x, 0) / max_constructor_points)
        else:
             race_data['ConstructorPointsIndex'] = 0.0
             
        race_data.rename(columns={'Abbreviation': 'Driver', 'TeamName': 'Team'}, inplace=True)
        
        mean_quali_time = race_data['QualifyingTime (s)'].mean()
        race_data['QualifyingTime (s)'].fillna(mean_quali_time, inplace=True)
        
        # Ensure no NaNs remain in core features, including the track and driver features
        core_features = ['Driver', 'Team', 'FinishingPosition', 'QualifyingTime (s)', 
                         'PointsIndex', 'ConstructorPointsIndex', 'GridPosition']
        core_features.extend(track_features.keys())
        driver_feature_keys = list(get_driver_features('VER').keys())  # Get a sample of driver feature keys
        core_features.extend(driver_feature_keys)
        race_data.dropna(subset=core_features, inplace=True)
        
        # Include track and driver features in the returned columns
        return_columns = ['Year', 'EventName', 'Driver', 'Team', 'QualifyingTime (s)', 
                          'PointsIndex', 'ConstructorPointsIndex', 'GridPosition', 'FinishingPosition']
        return_columns.extend(track_features.keys())
        return_columns.extend(driver_feature_keys)
        
        return race_data[return_columns]
        
    except Exception as e:
        print(f"Error processing {year} {event_identifier}: {e}")
        return None

# --- Configuration ---
TARGET_YEAR = 2025
TARGET_GP_NAME = 'Japanese' # Example: 'Monaco', 'British', 'Saudi Arabian Grand Prix'
HISTORICAL_YEARS = range(TARGET_YEAR - 1, 2017, -1) # e.g., 2024 down to 2019

# --- Data Gathering ---
all_race_data = []

# 1. Get Target GP Info
try:
    schedule_target_year = fastf1.get_event_schedule(TARGET_YEAR, include_testing=False)
    target_event = schedule_target_year[schedule_target_year['EventName'].str.contains(TARGET_GP_NAME, case=False, na=False)]
    
    if target_event.empty:
        raise ValueError(f"Target GP '{TARGET_GP_NAME}' not found in {TARGET_YEAR} schedule.")
        
    target_round = target_event['RoundNumber'].iloc[0]
    target_venue = target_event['Location'].iloc[0]
    target_event_name = target_event['EventName'].iloc[0] # Official name for consistency
    print(f"Target Event: {TARGET_YEAR} {target_event_name} (Round {target_round}, Venue: {target_venue})")

except Exception as e:
    raise ValueError(f"Error finding target GP info: {e}")

# 2. Get Data from Current Season (up to target GP)
print(f"\n--- Fetching {TARGET_YEAR} data before Round {target_round} ---")
schedule_current_year_filtered = schedule_target_year[schedule_target_year['RoundNumber'] < target_round]
for index, race in schedule_current_year_filtered.iterrows():
    print(f"Processing {TARGET_YEAR} {race['EventName']}...")
    data = get_race_data(TARGET_YEAR, race['EventName'])
    if data is not None:
        all_race_data.append(data)

# 3. Get Historical Data
print(f"\n--- Fetching Historical Data ---")
for year in HISTORICAL_YEARS:
    try:
        schedule_hist = fastf1.get_event_schedule(year, include_testing=False)
        print(f"Processing all races for {year}...")
        # Iterate through all races in the historical year's schedule
        for index, race_hist in schedule_hist.iterrows():
            event_name_hist = race_hist['EventName']
            print(f"  Processing {year} {event_name_hist}...")
            data = get_race_data(year, event_name_hist)
            if data is not None:
                all_race_data.append(data)
    except Exception as e:
        print(f"Error getting schedule or processing races for {year}: {e}")

# 4. Combine Data
if not all_race_data:
    raise ValueError("No data could be collected. Check FastF1 setup and GP/Year validity.")

combined_data = pd.concat(all_race_data, ignore_index=True)

# Drop the RoundNumber column
if 'RoundNumber' in combined_data.columns:
    combined_data.drop(columns=['RoundNumber'], inplace=True)
    print("Dropped 'RoundNumber' column.")
else:
    print("'RoundNumber' column not found in combined_data.")

print(f"\n--- Combined Data Shape after dropping RoundNumber: {combined_data.shape} ---")
print(combined_data.head())
print(combined_data.info())
print(combined_data.describe())

# Export the combined dataset to CSV
try:
    csv_filename = 'combined_f1_data.csv'
    combined_data.to_csv(csv_filename, index=False)
    print(f"\n--- Full Combined Dataset exported to {csv_filename} ---")
except Exception as e:
    print(f"\nError exporting data to CSV: {e}")

# 5. Prepare Data for Target Race Prediction
print(f"\n--- Preparing data for {TARGET_YEAR} {target_event_name} prediction ---")
try:
    # Get Qualifying data for the actual target race
    target_q_session = fastf1.get_session(TARGET_YEAR, target_event_name, 'Q')
    target_q_session.load(laps=False, telemetry=False, weather=False, messages=False)
    target_q_results = target_q_session.results
    
    if target_q_results is None or target_q_results.empty:
         raise ValueError(f"Could not load Qualifying results for target event: {TARGET_YEAR} {target_event_name}")
         
    target_q_results['QualifyingTime'] = target_q_results[['Q1', 'Q2', 'Q3']].min(axis=1)
    target_q_results['QualifyingTime (s)'] = target_q_results['QualifyingTime'].dt.total_seconds()
    
    # Get points index before the target race using the updated function
    target_driver_standings, target_max_driver_points, target_constructor_standings, target_max_constructor_points = get_standings_before_round(TARGET_YEAR, target_round)
    
    # Create DataFrame for prediction
    predict_df = target_q_results[['Abbreviation', 'TeamName', 'QualifyingTime (s)']].copy()
    predict_df.rename(columns={'Abbreviation': 'Driver', 'TeamName': 'Team'}, inplace=True)
    
    # Calculate Driver Points Index for prediction set
    if target_max_driver_points > 0:
         predict_df['PointsIndex'] = predict_df['Driver'].apply(lambda x: target_driver_standings.get(x, 0) / target_max_driver_points)
    else:
         predict_df['PointsIndex'] = 0.0
         
    # Calculate Constructor Points Index for prediction set
    if target_max_constructor_points > 0:
         predict_df['ConstructorPointsIndex'] = predict_df['Team'].apply(lambda x: target_constructor_standings.get(x, 0) / target_max_constructor_points)
    else:
         predict_df['ConstructorPointsIndex'] = 0.0
         
    # Handle potential missing Quali times in prediction set (e.g., pit lane start)
    # Use mean from the combined *training* data to fill
    mean_quali_train = combined_data['QualifyingTime (s)'].mean()
    predict_df['QualifyingTime (s)'].fillna(mean_quali_train, inplace=True)
    
    # Sort the qualifying results by time to determine grid positions
    predict_df = predict_df.sort_values('QualifyingTime (s)')
    predict_df['GridPosition'] = range(1, len(predict_df) + 1)
    
    # Add track features to the prediction dataframe for the target race
    target_track_features = get_track_features(target_event_name)
    for feature_name, feature_value in target_track_features.items():
        predict_df[feature_name] = feature_value
    
    # Add driver features to the prediction dataframe for the target race
    for idx, row in predict_df.iterrows():
        driver_code = row['Driver']
        driver_features = get_driver_features(driver_code)
        for feature_name, feature_value in driver_features.items():
            predict_df.at[idx, feature_name] = feature_value
    
    # Ensure no NaNs in prediction features
    predict_df.dropna(subset=['QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex'], inplace=True)
    
    # Keep track of drivers for final output
    predict_drivers = predict_df['Driver'].tolist()
    
    print(f"Prediction input data shape: {predict_df.shape}")
    print(predict_df)
    
except Exception as e:
    raise ValueError(f"Error preparing prediction data: {e}")

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	

Target Event: 2025 Japanese Grand Prix (Round 3, Venue: Suzuka)

--- Fetching 2025 data before Round 3 ---
Processing 2025 Australian Grand Prix...
Processing 2025 Chinese Grand Prix...


core           INFO 	Loading data for Chinese Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['81', '63', '4', '1', '44', '16', '6', '12', '22', '23', '31', '27', '14', '18', '55', '10', '87', '7', '5', '30']
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['81', '63', '4', '1', '44', '16', '6', '12', '22', '23', '31', '27', '14', '18', '55', '10', '87', '7', '5', '30']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55

Track characteristics not found for 'Chinese Grand Prix'. Using default values.

--- Fetching Historical Data ---


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2',

...

  Processing 2018 Brazilian Grand Prix...
Driver characteristics not found for 'RAI'. Using default values.
Driver characteristics not found for 'RIC'. Using default values.
Driver characteristics not found for 'BOT'. Using default values.
Driver characteristics not found for 'VET'. Using default values.
Driver characteristics not found for 'GRO'. Using default values.
Driver characteristics not found for 'MAG'. Using default values.
Driver characteristics not found for 'PER'. Using default values.
Driver characteristics not found for 'HAR'. Using default values.
Driver characteristics not found for 'VAN'. Using default values.
Driver characteristics not found for 'SIR'. Using default values.
Driver characteristics not found for 'ERI'. Using default values.


core           INFO 	Finished loading data for 20 drivers: ['3', '77', '7', '44', '33', '27', '14', '5', '55', '20', '31', '11', '2', '18', '35', '9', '8', '10', '16', '28']
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '7', '11', '5', '55', '16', '14', '18', '2', '28', '9', '10', '20', '77', '8', '33', '3', '27', '31', '35']
core           INFO 	Finished loading data for 20 drivers: ['44', '7', '11', '5', '55', '16', '14', '18', '2', '28', '9', '10', '20', '77', '8', '33', '3', '27', '31', '35']
core           INFO 	Loading data for Spanish Grand Prix - Race [v3.5.3]
req            INFO 	Using cach

  Processing 2018 Abu Dhabi Grand Prix...
Driver characteristics not found for 'VET'. Using default values.
Driver characteristics not found for 'RIC'. Using default values.
Driver characteristics not found for 'BOT'. Using default values.
Driver characteristics not found for 'PER'. Using default values.
Driver characteristics not found for 'GRO'. Using default values.
Driver characteristics not found for 'MAG'. Using default values.
Driver characteristics not found for 'HAR'. Using default values.
Driver characteristics not found for 'VAN'. Using default values.
Driver characteristics not found for 'SIR'. Using default values.
Driver characteristics not found for 'ERI'. Using default values.
Driver characteristics not found for 'RAI'. Using default values.


core           INFO 	Finished loading data for 20 drivers: ['3', '77', '7', '44', '33', '27', '14', '5', '55', '20', '31', '11', '2', '18', '35', '9', '8', '10', '16', '28']
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.5.3]
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '7', '11', '5', '55', '16', '14', '18', '2', '28', '9', '10', '20', '77', '8', '33', '3', '27', '31', '35']
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core           INFO 	Finished loading data for 20 drivers: ['44', '7', '11', '5', '55', '16', '14', '18', '2', '28', '9', '10', '20', '77', '8', '33', '3', '27', '31', '35']
core           INFO 	Loading data for Spanish Grand Prix - Race [v3.5.3]
req            INFO 	Using cach

'RoundNumber' column not found in combined_data.

--- Combined Data Shape after dropping RoundNumber: (3016, 19) ---
   Year              EventName Driver             Team  QualifyingTime (s)  \
0  2025  Australian Grand Prix    NOR          McLaren              75.096   
1  2025  Australian Grand Prix    VER  Red Bull Racing              75.481   
2  2025  Australian Grand Prix    RUS         Mercedes              75.546   
3  2025  Australian Grand Prix    ANT         Mercedes              76.525   
4  2025  Australian Grand Prix    ALB         Williams              75.737   

   PointsIndex  ConstructorPointsIndex  GridPosition  FinishingPosition  \
0          0.0                     0.0             1                  1   
1          0.0                     0.0             3                  2   
2          0.0                     0.0             4                  3   
3          0.0                     0.0            16                  4   
4          0.0                     0.0 

In [10]:

# Define all the features we'll use for the prediction model
# Include the new driver-specific features
features = ['GridPosition', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex', 
           'HighDownforce', 'HighSpeed', 'TireWear', 'OvertakingDifficulty',
           'WetWeatherSkill', 'QualifyingPace', 'RaceCraft', 'Consistency', 
           'Aggression', 'TireManagement']
target = 'FinishingPosition'

# Separate features (X) and target (y) from the combined historical/current data
X = combined_data[features].copy()
y = combined_data[target].copy()


# --- Prepare Prediction Data ---
predict_X = predict_df[features].copy()

# Add track features to the prediction data for the target race
target_track_features = get_track_features(target_event_name)
for feature_name, feature_value in target_track_features.items():
    predict_X[feature_name] = feature_value

print(predict_X.head())

# --- Train Gradient Boosting Model ---
# Split the historical/current season data for evaluation (optional but good practice)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using Gradient Boosting - a reasonably powerful standard model
# Hyperparameters can be tuned for better performance, but these are sensible defaults.
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
print("\n--- Training Model ---")
model.fit(X_train, y_train)

# --- Predict Finishing Positions for Target Race ---
print(f"\n--- Predicting for {TARGET_YEAR} {target_event_name} ---")
predicted_scores = model.predict(predict_X)

# --- Create Ranked Results ---
# Create a DataFrame with drivers and their raw predicted scores (lower score is better)
results_df = pd.DataFrame({
    'Driver': predict_drivers, 
    'PredictedScore': predicted_scores
})

# Sort drivers by the predicted score (ascending - lower score means better predicted position)
results_df = results_df.sort_values(by='PredictedScore')

# Assign ranks 1, 2, 3... based on the sorted order
results_df['PredictedFinishingPosition'] = np.arange(1, len(results_df) + 1)

# Print final ranked predictions
print(f"\n🏁 Predicted Finishing Order for {TARGET_YEAR} {target_event_name} 🏁\n")
print(results_df[['Driver', 'PredictedFinishingPosition']].to_string(index=False))

# --- Evaluate Model on Test Set ---
print("\n--- Evaluating Model Performance ---")
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f"🔍 Model Evaluation (Mean Absolute Error on test set): {mae:.2f} positions")
print("(Lower MAE is better. This indicates the average error in predicted position on unseen historical data)")

# --- Feature Importance ---
print("\n📊 Feature Importances:")
try:
    feature_names = X.columns
    importances = model.feature_importances_
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    print(importance_df.to_string(index=False))
except Exception as e:
    print(f"Could not calculate or display feature importances: {e}")

    GridPosition  QualifyingTime (s)  PointsIndex  ConstructorPointsIndex  \
1              1              86.983     0.697674                0.428571   
4              2              86.995     1.000000                1.000000   
81             3              87.027     0.627907                1.000000   
16             4              87.299     0.093023                0.071429   
63             5              87.318     0.697674                0.714286   

    HighDownforce  HighSpeed  TireWear  OvertakingDifficulty  WetWeatherSkill  \
1             0.8        0.8       0.7                   0.7             0.95   
4             0.8        0.8       0.7                   0.7             0.85   
81            0.8        0.8       0.7                   0.7             0.82   
16            0.8        0.8       0.7                   0.7             0.82   
63            0.8        0.8       0.7                   0.7             0.80   

    QualifyingPace  RaceCraft  Consistency  Aggres